<pre>
*****PySpark DF Question******
Filename: covid19.txt
1) Convert both the date columns to (yyyy-MM-dd) format.
2) Have an extra column and populate the number of days between these two days.
3) Partition the data by year columns and write to HDFS in parquet/avro/json format.
</pre>

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("yettodecide").master("local").getOrCreate()

In [20]:
df = spark.read.format("csv")\
          .option("header",True)\
          .option("InferSchema",True)\
          .load("C:\\Personal\\Projects\\Spark\\dataset\\covid19.txt")

In [21]:
df.show(2,truncate=False)

+-------+----+----------+--------+----------+---+---+---+---+---------+---------+
|D      |Y   |Dt        |Wkd     |CM        |C  |Com|TM |M  |V        |CL       |
+-------+----+----------+--------+----------+---+---+---+---+---------+---------+
|Exports|2015|31/01/2015|Saturday|01/02/2020|All|All|All|$  |257000000|257000000|
|Exports|2015|01/02/2015|Sunday  |02/02/2020|All|All|All|$  |123000000|380000000|
+-------+----+----------+--------+----------+---+---+---+---+---------+---------+
only showing top 2 rows



In [22]:
from pyspark.sql.functions import col,date_format, to_timestamp, datediff

In [23]:
df = df.withColumn("Dt", date_format(to_timestamp(col("Dt"),"dd/MM/yyyy"), "yyyy-MM-dd") )\
  .withColumn("CM", date_format(to_timestamp(col("CM"),"dd/MM/yyyy"), "yyyy-MM-dd") )\
  .withColumn("DiffDate" , datediff(col("CM"),col("DT")) )

In [24]:
df.show(5)

+-------+----+----------+---------+----------+---+---+---+---+---------+---------+--------+
|      D|   Y|        Dt|      Wkd|        CM|  C|Com| TM|  M|        V|       CL|DiffDate|
+-------+----+----------+---------+----------+---+---+---+---+---------+---------+--------+
|Exports|2015|2015-01-31| Saturday|2020-02-01|All|All|All|  $|257000000|257000000|    1827|
|Exports|2015|2015-02-01|   Sunday|2020-02-02|All|All|All|  $|123000000|380000000|    1827|
|Exports|2015|2015-02-02|   Monday|2020-02-03|All|All|All|  $|176000000|556000000|    1827|
|Exports|2015|2015-02-03|  Tuesday|2020-02-04|All|All|All|  $|115000000|671000000|    1827|
|Exports|2015|2015-02-04|Wednesday|2020-02-05|All|All|All|  $| 74000000|746000000|    1827|
+-------+----+----------+---------+----------+---+---+---+---+---------+---------+--------+
only showing top 5 rows



In [25]:
df.write.format('json')\
        .option("header",True)\
        .partitionBy("Y")\
        .mode("overwrite")\
        .save("C:\\Personal\\Projects\\Spark\\dataset\\5th_assignment.csv")